# Solar Abundance Analysis

## Author(s): Sven Buder (SB, WG4)

### History:
180926 SB Created

In [192]:
# Preamble for notebook 

# Compatibility with Python 3
from __future__ import (absolute_import, division, print_function)

try:
    %matplotlib inline
    %config InlineBackend.figure_format='retina'
except:
    pass

# Basic packages
import numpy as np
np.seterr(divide='ignore', invalid='ignore')
import os
import sys
import collections
import glob
import pickle
import pandas

# Packages to work with FITS and (IDL) SME.out files
import astropy.io.fits as pyfits
import astropy.table as table
from astropy.table import Table, join
from scipy.io.idl import readsav

# Matplotlib and associated packages for plotting
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from matplotlib.transforms import Bbox,TransformedBbox
from matplotlib.image import BboxImage
from matplotlib.legend_handler import HandlerBase
from matplotlib._png import read_png
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib.colors import ListedColormap
import matplotlib.colors as colors

params = {
#     'font.family'        : 'sans',
#     'font.size'          : 17,
#     'axes.labelsize'     : 20,
#     'ytick.labelsize'    : 16,
#     'xtick.labelsize'    : 16,
#     'legend.fontsize'    : 20,
    'text.usetex'        : True, 
    'text.latex.preamble': [r'\usepackage{upgreek}', r'\usepackage{amsmath}'],
    }   
plt.rcParams.update(params)

_parula_data = [[0.2081, 0.1663, 0.5292], 
                [0.2116238095, 0.1897809524, 0.5776761905], 
                [0.212252381, 0.2137714286, 0.6269714286], 
                [0.2081, 0.2386, 0.6770857143], 
                [0.1959047619, 0.2644571429, 0.7279], 
                [0.1707285714, 0.2919380952, 0.779247619], 
                [0.1252714286, 0.3242428571, 0.8302714286], 
                [0.0591333333, 0.3598333333, 0.8683333333], 
                [0.0116952381, 0.3875095238, 0.8819571429], 
                [0.0059571429, 0.4086142857, 0.8828428571], 
                [0.0165142857, 0.4266, 0.8786333333], 
                [0.032852381, 0.4430428571, 0.8719571429], 
                [0.0498142857, 0.4585714286, 0.8640571429], 
                [0.0629333333, 0.4736904762, 0.8554380952], 
                [0.0722666667, 0.4886666667, 0.8467], 
                [0.0779428571, 0.5039857143, 0.8383714286], 
                [0.079347619, 0.5200238095, 0.8311809524], 
                [0.0749428571, 0.5375428571, 0.8262714286], 
                [0.0640571429, 0.5569857143, 0.8239571429], 
                [0.0487714286, 0.5772238095, 0.8228285714], 
                [0.0343428571, 0.5965809524, 0.819852381], 
                [0.0265, 0.6137, 0.8135], 
                [0.0238904762, 0.6286619048, 0.8037619048], 
                [0.0230904762, 0.6417857143, 0.7912666667], 
                [0.0227714286, 0.6534857143, 0.7767571429], 
                [0.0266619048, 0.6641952381, 0.7607190476], 
                [0.0383714286, 0.6742714286, 0.743552381], 
                [0.0589714286, 0.6837571429, 0.7253857143], 
                [0.0843, 0.6928333333, 0.7061666667], 
                [0.1132952381, 0.7015, 0.6858571429], 
                [0.1452714286, 0.7097571429, 0.6646285714], 
                [0.1801333333, 0.7176571429, 0.6424333333], 
                [0.2178285714, 0.7250428571, 0.6192619048], 
                [0.2586428571, 0.7317142857, 0.5954285714], 
                [0.3021714286, 0.7376047619, 0.5711857143], 
                [0.3481666667, 0.7424333333, 0.5472666667], 
                [0.3952571429, 0.7459, 0.5244428571], 
                [0.4420095238, 0.7480809524, 0.5033142857], 
                [0.4871238095, 0.7490619048, 0.4839761905], 
                [0.5300285714, 0.7491142857, 0.4661142857], 
                [0.5708571429, 0.7485190476, 0.4493904762],
                [0.609852381, 0.7473142857, 0.4336857143], 
                [0.6473, 0.7456, 0.4188], 
                [0.6834190476, 0.7434761905, 0.4044333333], 
                [0.7184095238, 0.7411333333, 0.3904761905], 
                [0.7524857143, 0.7384, 0.3768142857], 
                [0.7858428571, 0.7355666667, 0.3632714286], 
                [0.8185047619, 0.7327333333, 0.3497904762], 
                [0.8506571429, 0.7299, 0.3360285714], 
                [0.8824333333, 0.7274333333, 0.3217], 
                [0.9139333333, 0.7257857143, 0.3062761905], 
                [0.9449571429, 0.7261142857, 0.2886428571], 
                [0.9738952381, 0.7313952381, 0.266647619], 
                [0.9937714286, 0.7454571429, 0.240347619], 
                [0.9990428571, 0.7653142857, 0.2164142857], 
                [0.9955333333, 0.7860571429, 0.196652381], 
                [0.988, 0.8066, 0.1793666667], 
                [0.9788571429, 0.8271428571, 0.1633142857], 
                [0.9697, 0.8481380952, 0.147452381], 
                [0.9625857143, 0.8705142857, 0.1309], 
                [0.9588714286, 0.8949, 0.1132428571], 
                [0.9598238095, 0.9218333333, 0.0948380952], 
                [0.9661, 0.9514428571, 0.0755333333], 
                [0.9763, 0.9831, 0.0538]]

parula = ListedColormap(_parula_data, name='parula')
parula_zero = _parula_data[0]
parula_0 = ListedColormap(_parula_data, name='parula_0')
parula_0.set_bad((1,1,1))
parula_r = ListedColormap(_parula_data[::-1], name='parula_r')

willi_blau = [0.0722666667, 0.4886666667, 0.8467]

In [193]:
single_spectrum = dict()
single_spectrum['directory'] = 'solar_abundances/single_spectrum/'
single_spectrum['field'] = 'gbs_lbol'
single_spectrum['sobject_id'] = '150405000901378'

combined_spectrum = dict()
combined_spectrum['directory'] = 'solar_abundances/combined_spectrum/'
combined_spectrum['field'] = 'solar_lbol'
combined_spectrum['sobject_id'] = 'super_combined'

def get_mode_sme(single_combined_dictionary, mode='Sp'):
    sme = readsav(single_combined_dictionary['directory']+single_combined_dictionary['field']+'_'+single_combined_dictionary['sobject_id']+'_DR3_'+mode+'_SME.out').sme[0]
    return sme

# Comparison with Asplund09 and APOGEE DR16

In [194]:
# Here we load the information from GALAH DR3
galah_dr3 = Table.read('../../catalogs/GALAH_DR3_main.fits')
sun_entry = galah_dr3['sobject_id'] == 150405000901378
galah_sun = galah_dr3[sun_entry][0]

In [195]:
galah_zeropoints = Table.read('galahdr3_abundance_zeropoints.fits',1)

In [196]:
galah = dict()
galah['teff'] = "{:.0f}".format(galah_sun['teff'])
galah['e_teff'] = "{:.0f}".format(galah_sun['e_teff'])

galah['logg'] = "{:.2f}".format(galah_sun['logg'])
galah['e_logg'] = "{:.2f}".format(galah_sun['e_logg'])

galah['m_h'] = "{:.2f}".format(galah_sun['fe_h_atmo'])
galah['e_m_h'] = "{:.2f}".format(galah_sun['e_fe_h_atmo'])

galah['fe_h'] = "{:.2f}".format(galah_sun['fe_h'])
galah['e_fe_h'] = "{:.2f}".format(galah_sun['e_fe_h'])

galah['mass'] = "{:.2f}".format(galah_sun['mass'])
galah['e_mass'] = '-'

galah['age'] = "{:.2f}".format(galah_sun['age'])
galah['e_age'] = '-'

galah['lbol'] = "{:.2f}".format(galah_sun['lbol'])
galah['e_lbol'] = '-'

galah['vmic'] = "{:.2f}".format(galah_sun['vmic'])
galah['e_vmic'] = '-'

galah['vbroad'] = "{:.2f}".format(galah_sun['vbroad'])
galah['e_vbroad'] = "{:.2f}".format(galah_sun['e_vbroad'])

galah['alpha_fe'] = "{:.2f}".format(galah_sun['alpha_fe'])
galah['e_alpha_fe'] = "{:.2f}".format(galah_sun['e_alpha_fe'])

for element in ['Li','C','O','Na','Mg','Al','Si','K','Ca','Sc','Ti','Ti2','V','Cr','Mn','Co','Ni','Cu','Zn','Rb','Sr','Y','Zr','Mo','Ru','Ba','La','Ce','Nd','Sm','Eu']:
    if np.isfinite(galah_sun[element+'_fe']):
        galah[element+'_fe'] = "{:.2f}".format(galah_sun[element+'_fe'])
        galah['e_'+element+'_fe'] = "{:.2f}".format(galah_sun['e_'+element+'_fe'])
    else:
        galah[element+'_fe'] = '-'
        galah['e_'+element+'_fe'] = '-'
        
for element in ['alpha_fe','Ti','Ti2','V','Co','Ni','Cu','Zn','Zr','Mo','Ru','La','Nd','Sm']:
    galah['A_'+element] = '-'
    galah['e_A_'+element] = '-'
        
for element in ['O','Na','Al','Si','Ca','Sc','Cr','Mn','Fe','Y','Ba']:
    try:
        galah['A_'+element] = "{:.2f}".format(galah_zeropoints['A_'+element][0])
        galah['e_A_'+element] = "{:.2f}".format(galah_zeropoints['A_'+element][1])
    except:
        print(element, 'not found')
        
for element in ['Li6708','C6588','Mg5711','K7699',
                'Ti4758','Ti4759','Ti4782','Ti4802','Ti4820','Ti5739',
                'Ti4720','Ti4765','Ti4799','Ti4866',
                'V4784','V4797','V4832',
                'Co5647','Co6490','Co6632','Co7713',
                'Ni5847','Ni6586','Cu5700','Cu5782',
                'Zn4722','Zn4811','Rb7800','Sr6550','Y4855','Y4884',
                'Zr4739','Zr4772','Zr4806','Zr4828','Zr5681',
                'Mo5858','Mo6619','La4749','La4804','La5806',
                'Ce4774','Nd4811','Nd5812','Sm4720','Sm4848','Eu6645'
    ]:
    galah['A_'+element] = "{:.2f}".format(galah_zeropoints['A_'+element][0])
    #galah['e_A_'+element[:-4]] = "{:.2f}".format(galah_zeropoints['A_'+element][1])


print(galah)

{'teff': '5779', 'e_teff': '62', 'logg': '4.42', 'e_logg': '0.14', 'm_h': '0.01', 'e_m_h': '0.06', 'fe_h': '0.00', 'e_fe_h': '0.06', 'mass': '0.97', 'e_mass': '-', 'age': '5.83', 'e_age': '-', 'lbol': '1.01', 'e_lbol': '-', 'vmic': '1.16', 'e_vmic': '-', 'vbroad': '6.52', 'e_vbroad': '2.03', 'alpha_fe': '-0.00', 'e_alpha_fe': '0.02', 'Li_fe': '-', 'e_Li_fe': '-', 'C_fe': '-', 'e_C_fe': '-', 'O_fe': '-0.17', 'e_O_fe': '0.06', 'Na_fe': '-0.10', 'e_Na_fe': '0.02', 'Mg_fe': '0.00', 'e_Mg_fe': '0.03', 'Al_fe': '-0.02', 'e_Al_fe': '0.03', 'Si_fe': '-0.02', 'e_Si_fe': '0.03', 'K_fe': '-0.09', 'e_K_fe': '0.04', 'Ca_fe': '-0.04', 'e_Ca_fe': '0.03', 'Sc_fe': '-0.00', 'e_Sc_fe': '0.02', 'Ti_fe': '-0.01', 'e_Ti_fe': '0.03', 'Ti2_fe': '-0.00', 'e_Ti2_fe': '0.03', 'V_fe': '-0.01', 'e_V_fe': '0.02', 'Cr_fe': '-0.00', 'e_Cr_fe': '0.00', 'Mn_fe': '-0.01', 'e_Mn_fe': '0.03', 'Co_fe': '-', 'e_Co_fe': '-', 'Ni_fe': '0.01', 'e_Ni_fe': '0.02', 'Cu_fe': '0.02', 'e_Cu_fe': '0.00', 'Zn_fe': '-0.00', 'e_Zn_fe':

In [197]:
# Here we save the information from  for solar constants as well as Asplund, 2009, ARAA, 47, 481.
asplund = dict()
asplund['teff'] = '5772'; asplund['e_teff'] = '-' # Prsa, 2016, AJ, 152, 2
asplund['logg'] = '4.438'; asplund['e_logg'] = '-' # Prsa, 2016, AJ, 152, 2
asplund['m_h'] = '-'; asplund['e_m_h'] = '-'
asplund['fe_h'] = '-'; asplund['e_fe_h'] = '-'
asplund['mass'] = '1.00'; asplund['e_mass'] = '-' # Prsa, 2016, AJ, 152, 2
asplund['lbol'] = '1.00'; asplund['e_lbol'] = '-' # Prsa, 2016, AJ, 152, 2
asplund['age'] = '4.57'; asplund['e_age'] = '0.11' # Bonanno 2002, A&A, 390, 1115
asplund['vmic'] = '1.74'; asplund['e_vmic'] = '-'
asplund['vbroad'] = '-'; asplund['e_vbroad'] = '-'

asplund['A_Li'] = '1.05'; asplund['e_A_Li'] = '0.10' # Asplund, 2009, ARAA, 47, 481.
asplund['A_C']  = '8.43'; asplund['e_A_C']  = '0.05' # Asplund, 2009, ARAA, 47, 481.
asplund['A_O']  = '8.69'; asplund['e_A_O']  = '0.05' # Asplund, 2009, ARAA, 47, 481.
asplund['A_Na'] = '6.24'; asplund['e_A_Na'] = '0.04' # Asplund, 2009, ARAA, 47, 481.
asplund['A_Mg'] = '7.60'; asplund['e_A_Mg'] = '0.04' # Asplund, 2009, ARAA, 47, 481.
asplund['A_Al'] = '6.45'; asplund['e_A_Al'] = '0.03' # Asplund, 2009, ARAA, 47, 481.
asplund['A_Si'] = '7.51'; asplund['e_A_Si'] = '0.03' # Asplund, 2009, ARAA, 47, 481.
asplund['A_K']  = '5.03'; asplund['e_A_K']  = '0.09' # Asplund, 2009, ARAA, 47, 481.
asplund['A_Ca'] = '6.34'; asplund['e_A_Ca'] = '0.04' # Asplund, 2009, ARAA, 47, 481.
asplund['A_Sc'] = '3.15'; asplund['e_A_Sc'] = '0.04' # Asplund, 2009, ARAA, 47, 481.
asplund['A_Ti'] = '4.95'; asplund['e_A_Ti'] = '0.05' # Asplund, 2009, ARAA, 47, 481.
asplund['A_Ti2']= '4.95'; asplund['e_A_Ti2']= '0.05' # Asplund, 2009, ARAA, 47, 481.
asplund['A_V']  = '3.93'; asplund['e_A_V']  = '0.08' # Asplund, 2009, ARAA, 47, 481.
asplund['A_Cr'] = '5.64'; asplund['e_A_Cr'] = '0.04' # Asplund, 2009, ARAA, 47, 481.
asplund['A_Mn'] = '5.43'; asplund['e_A_Mn'] = '0.04' # Asplund, 2009, ARAA, 47, 481.
asplund['A_Fe'] = '7.50'; asplund['e_A_Fe'] = '0.04' # Asplund, 2009, ARAA, 47, 481.
asplund['A_Co'] = '4.99'; asplund['e_A_Co'] = '0.07' # Asplund, 2009, ARAA, 47, 481.
asplund['A_Ni'] = '6.22'; asplund['e_A_Ni'] = '0.04' # Asplund, 2009, ARAA, 47, 481.
asplund['A_Cu'] = '4.19'; asplund['e_A_Cu'] = '0.04' # Asplund, 2009, ARAA, 47, 481.
asplund['A_Zn'] = '4.56'; asplund['e_A_Zn'] = '0.05' # Asplund, 2009, ARAA, 47, 481.
asplund['A_Rb'] = '2.52'; asplund['e_A_Rb'] = '0.10' # Asplund, 2009, ARAA, 47, 481.
asplund['A_Sr'] = '2.87'; asplund['e_A_Sr'] = '0.07' # Asplund, 2009, ARAA, 47, 481.
asplund['A_Y']  = '2.21'; asplund['e_A_Y']  = '0.05' # Asplund, 2009, ARAA, 47, 481.
asplund['A_Zr'] = '2.58'; asplund['e_A_Zr'] = '0.04' # Asplund, 2009, ARAA, 47, 481.
asplund['A_Mo'] = '1.88'; asplund['e_A_Mo'] = '0.08' # Asplund, 2009, ARAA, 47, 481.
asplund['A_Ru'] = '1.75'; asplund['e_A_Ru'] = '0.08' # Asplund, 2009, ARAA, 47, 481.
asplund['A_Ba'] = '2.18'; asplund['e_A_Ba'] = '0.09' # Asplund, 2009, ARAA, 47, 481.
asplund['A_La'] = '1.10'; asplund['e_A_La'] = '0.04' # Asplund, 2009, ARAA, 47, 481.
asplund['A_Ce'] = '1.58'; asplund['e_A_Ce'] = '0.04' # Asplund, 2009, ARAA, 47, 481.
asplund['A_Nd'] = '1.42'; asplund['e_A_Nd'] = '0.04' # Asplund, 2009, ARAA, 47, 481.
asplund['A_Sm'] = '0.96'; asplund['e_A_Sm'] = '0.04' # Asplund, 2009, ARAA, 47, 481.
asplund['A_Eu'] = '0.52'; asplund['e_A_Eu'] = '0.04' # Asplund, 2009, ARAA, 47, 481.

print(asplund)

{'teff': '5772', 'e_teff': '-', 'logg': '4.438', 'e_logg': '-', 'm_h': '-', 'e_m_h': '-', 'fe_h': '-', 'e_fe_h': '-', 'mass': '1.00', 'e_mass': '-', 'lbol': '1.00', 'e_lbol': '-', 'age': '4.57', 'e_age': '0.11', 'vmic': '1.74', 'e_vmic': '-', 'vbroad': '-', 'e_vbroad': '-', 'A_Li': '1.05', 'e_A_Li': '0.10', 'A_C': '8.43', 'e_A_C': '0.05', 'A_O': '8.69', 'e_A_O': '0.05', 'A_Na': '6.24', 'e_A_Na': '0.04', 'A_Mg': '7.60', 'e_A_Mg': '0.04', 'A_Al': '6.45', 'e_A_Al': '0.03', 'A_Si': '7.51', 'e_A_Si': '0.03', 'A_K': '5.03', 'e_A_K': '0.09', 'A_Ca': '6.34', 'e_A_Ca': '0.04', 'A_Sc': '3.15', 'e_A_Sc': '0.04', 'A_Ti': '4.95', 'e_A_Ti': '0.05', 'A_Ti2': '4.95', 'e_A_Ti2': '0.05', 'A_V': '3.93', 'e_A_V': '0.08', 'A_Cr': '5.64', 'e_A_Cr': '0.04', 'A_Mn': '5.43', 'e_A_Mn': '0.04', 'A_Fe': '7.50', 'e_A_Fe': '0.04', 'A_Co': '4.99', 'e_A_Co': '0.07', 'A_Ni': '6.22', 'e_A_Ni': '0.04', 'A_Cu': '4.19', 'e_A_Cu': '0.04', 'A_Zn': '4.56', 'e_A_Zn': '0.05', 'A_Rb': '2.52', 'e_A_Rb': '0.10', 'A_Sr': '2.87', '

In [198]:
# Here we save the information from APOGEE DR16, reference: arXiv:1912.02905
apogee_dr16 = Table.read('../../../Surveys/APOGEE_DR16.fits')
sun_entry = apogee_dr16['APOGEE_ID'] == 'VESTA             '
apogee_sun = apogee_dr16[sun_entry][0]

In [199]:
apogee = dict()
apogee['teff'] = "{:.0f}".format(apogee_sun['TEFF'])
apogee['e_teff'] = "{:.0f}".format(apogee_sun['TEFF_ERR'])

apogee['logg'] = "{:.2f}".format(apogee_sun['LOGG'])
apogee['e_logg'] = "{:.2f}".format(apogee_sun['LOGG_ERR'])

apogee['m_h'] = "{:.2f}".format(apogee_sun['M_H'])
apogee['e_m_h'] = "{:.2f}".format(apogee_sun['M_H_ERR'])

apogee['fe_h'] = "{:.2f}".format(apogee_sun['FE_H'])
apogee['e_fe_h'] = "{:.2f}".format(apogee_sun['FE_H_ERR'])

apogee['mass'] = '-'
apogee['e_mass'] = '-'

apogee['age'] = '-'
apogee['e_age'] = '-'

apogee['lbol'] = '-'
apogee['e_lbol'] = '-'

apogee['vmic'] = "{:.2f}".format(apogee_sun['VMICRO'])
apogee['e_vmic'] = '-'

apogee['vbroad'] = "{:.2f}".format(np.sqrt(apogee_sun['VMACRO']**2+apogee_sun['VSINI']**2))
apogee['e_vbroad'] = '-'

apogee['Fe_M'] = '-0.004'

apogee['alpha_fe'] = "{:.2f}".format(apogee_sun['ALPHA_M']+apogee_sun['M_H']+apogee_sun['FE_H'])
apogee['e_alpha_fe'] = "{:.2f}".format(apogee_sun['ALPHA_M_ERR'])

for element in ['Li','C','O','Na','Mg','Al','Si','K','Ca','Sc','Ti','Ti2','V','Cr','Mn','Co','Ni','Cu','Zn','Rb','Sr','Y','Zr','Mo','Ru','Ba','La','Ce','Nd','Sm','Eu']:
    try:
        apogee[element+'_fe'] = "{:.2f}".format(apogee_sun[element.upper()+'_FE'])
        apogee['e_'+element+'_fe'] = "{:.2f}".format(apogee_sun[element.upper()+'_FE_ERR'])
    except:
        if element == 'Ti2':
            apogee[element+'_fe'] = "{:.2f}".format(apogee_sun['TIII_FE'])
            apogee['e_'+element+'_fe'] = "{:.2f}".format(apogee_sun['TIII_FE_ERR'])
        elif element == 'C':
            apogee[element+'_fe'] = "{:.2f}".format(apogee_sun['CI_FE'])
            apogee['e_'+element+'_fe'] = "{:.2f}".format(apogee_sun['CI_FE_ERR'])
        else:
            apogee[element+'_fe'] = '-'; apogee['e_'+element+'_fe'] = '-'
    if apogee[element+'_fe'] == '-9999.99':
        apogee[element+'_fe'] = '-'; apogee['e_'+element+'_fe'] = '-'

apogee['O_fe']  = "{:.2f}".format(0.045 - (-0.004)) # [O/M] - [Fe/M] from https://data.sdss.org/sas/dr16/apogee/spectro/aspcap/r12/l33/allStar-r12-l33.html
apogee['Na_fe'] = "{:.2f}".format(-0.017 - (-0.004)) # [O/M] - [Fe/M] from https://data.sdss.org/sas/dr16/apogee/spectro/aspcap/r12/l33/allStar-r12-l33.html
apogee['V_fe']  = "{:.2f}".format(-0.024 - (-0.004)) # [O/M] - [Fe/M] from https://data.sdss.org/sas/dr16/apogee/spectro/aspcap/r12/l33/allStar-r12-l33.html
apogee['Ce_fe'] = "{:.2f}".format(-0.116 - (-0.004)) # [O/M] - [Fe/M] from https://data.sdss.org/sas/dr16/apogee/spectro/aspcap/r12/l33/allStar-r12-l33.html

print(apogee)

{'teff': '5712', 'e_teff': '115', 'logg': '4.40', 'e_logg': '0.08', 'm_h': '0.00', 'e_m_h': '0.01', 'fe_h': '-0.00', 'e_fe_h': '0.01', 'mass': '-', 'e_mass': '-', 'age': '-', 'e_age': '-', 'lbol': '-', 'e_lbol': '-', 'vmic': '0.94', 'e_vmic': '-', 'vbroad': '5.85', 'e_vbroad': '-', 'Fe_M': '-0.004', 'alpha_fe': '-0.01', 'e_alpha_fe': '0.01', 'Li_fe': '-', 'e_Li_fe': '-', 'C_fe': '0.02', 'e_C_fe': '0.02', 'O_fe': '0.05', 'e_O_fe': '-', 'Na_fe': '-0.01', 'e_Na_fe': '-', 'Mg_fe': '-0.00', 'e_Mg_fe': '0.01', 'Al_fe': '0.01', 'e_Al_fe': '0.02', 'Si_fe': '-0.00', 'e_Si_fe': '0.01', 'K_fe': '-0.07', 'e_K_fe': '0.03', 'Ca_fe': '-0.01', 'e_Ca_fe': '0.02', 'Sc_fe': '-', 'e_Sc_fe': '-', 'Ti_fe': '-0.02', 'e_Ti_fe': '0.05', 'Ti2_fe': '0.11', 'e_Ti2_fe': '0.09', 'V_fe': '-0.02', 'e_V_fe': '-', 'Cr_fe': '0.04', 'e_Cr_fe': '0.06', 'Mn_fe': '0.05', 'e_Mn_fe': '0.02', 'Co_fe': '0.29', 'e_Co_fe': '0.14', 'Ni_fe': '0.02', 'e_Ni_fe': '0.02', 'Cu_fe': '-0.05', 'e_Cu_fe': '0.07', 'Zn_fe': '-', 'e_Zn_fe': '-

In [200]:
galah_dr3['TMASS_ID'] = galah_dr3['star_id']
galah_apogee = table.join(galah_dr3, apogee_dr16, keys='TMASS_ID')

In [201]:
delta_galah_apogee = dict()

galah_apogee['ALPHA_FE'] = galah_apogee['ALPHA_M'] + galah_apogee['M_H'] - galah_apogee['FE_H']

for element in ['teff','logg','m_h','fe_h','mass','age','lbol','vmic','vbroad','alpha_fe']:
    
    if element in ['teff','logg','m_h','fe_h','mass','age','lbol','vmic','vbroad']:
        delta_galah_apogee[element] = '-'
    else:
        
        if element == 'alpha_fe': apogee_element = 'ALPHA_FE'

        #ALL
        good_selection = (
            # good galah selection
            (galah_apogee['flag_sp'] == 0) &
            np.isfinite(galah_apogee[element]) &
            (galah_apogee['snr_c2_iraf'] > 100) &
            # good apogee selection
            (galah_apogee['ASPCAPFLAG'] == 0) &
            (np.abs(galah_apogee[apogee_element]) < 9997)
        )
        
        delta_galah_apogee[element] = r'$'+"{:.2f}".format(np.mean(galah_apogee[element][good_selection] - galah_apogee[apogee_element][good_selection]))+' \pm '+"{:.2f}".format(np.std(galah_apogee[element][good_selection] - galah_apogee[apogee_element][good_selection]))+'$'

        # DWARFS
        dwarf_selection = (
            # good galah selection
            (galah_apogee['flag_sp'] == 0) &
            np.isfinite(galah_apogee[element]) &
            (galah_apogee['snr_c2_iraf'] > 100) &
            # good apogee selection
            (galah_apogee['ASPCAPFLAG'] == 0) &
            (np.abs(galah_apogee[apogee_element]) < 9997) &
            # dwarfs
            (
                (galah_apogee['teff'] > 5500) | 
                (galah_apogee['logg'] > 3.8)
            )
        )
        
        delta_galah_apogee['dwarfs_'+element] = r'$'+"{:.2f}".format(np.mean(galah_apogee[element][dwarf_selection] - galah_apogee[apogee_element][dwarf_selection]))+' \pm '+"{:.2f}".format(np.std(galah_apogee[element][dwarf_selection] - galah_apogee[apogee_element][dwarf_selection]))+'$'

        # GIANTS
        
        giant_selection = (
            # good galah selection
            (galah_apogee['flag_sp'] == 0) &
            np.isfinite(galah_apogee[element]) &
            (galah_apogee['snr_c2_iraf'] > 100) &
            # good apogee selection
            (galah_apogee['ASPCAPFLAG'] == 0) &
            (np.abs(galah_apogee[apogee_element]) < 9997) &
            # giants
            (
                (galah_apogee['teff'] <= 5500) &
                (galah_apogee['logg'] <= 3.8)
            )
        )
        
        delta_galah_apogee['giants_'+element] = r'$'+"{:.2f}".format(np.mean(galah_apogee[element][giant_selection] - galah_apogee[apogee_element][giant_selection]))+' \pm '+"{:.2f}".format(np.std(galah_apogee[element][giant_selection] - galah_apogee[apogee_element][giant_selection]))+'$'

        
for element in ['Li','C','O','Na','Mg','Al','Si','K','Ca','Sc','Ti','Ti2','V','Cr','Mn','Co','Ni','Cu','Zn','Rb','Sr','Y','Zr','Mo','Ru','Ba','La','Ce','Nd','Sm','Eu']:
    
    if element in ['Li','Sc','Rb','Nd','Zn','Sr','Y','Zr','Mo','Ru','Ba','La','Sm','Eu']:
        delta_galah_apogee[element+'_fe'] = '-'
        delta_galah_apogee['dwarfs_'+element+'_fe'] = '-'
        delta_galah_apogee['giants_'+element+'_fe'] = '-'
    else:
        
        apogee_element = element.upper()
        if element == 'Ti2':
            apogee_element = 'TIII'

        good_selection = (
            # good galah selection
            (galah_apogee['flag_sp'] == 0) &
            np.isfinite(galah_apogee[element+'_fe']) &
            (galah_apogee['snr_c2_iraf'] > 100) &
            (galah_apogee['flag_'+element+'_fe'] == 0) &
            # good apogee selection
            (galah_apogee['ASPCAPFLAG'] == 0) &
            (galah_apogee[apogee_element+'_FE_FLAG'] == 0) &
            (galah_apogee[apogee_element+'_FE'] > -5)
        )        

        dwarf_selection = (
            # good galah selection
            (galah_apogee['flag_sp'] == 0) &
            np.isfinite(galah_apogee[element+'_fe']) &
            (galah_apogee['snr_c2_iraf'] > 100) &
            (galah_apogee['flag_'+element+'_fe'] == 0) &
            # good apogee selection
            (galah_apogee['ASPCAPFLAG'] == 0) &
            (galah_apogee[apogee_element+'_FE_FLAG'] == 0) &
            (galah_apogee[apogee_element+'_FE'] > -5) &
            # dwarfs
            (
                (galah_apogee['teff'] > 5500) |
                (galah_apogee['logg'] > 3.8)
            )
        )
            
        giant_selection = (
            # good galah selection
            (galah_apogee['flag_sp'] == 0) &
            np.isfinite(galah_apogee[element+'_fe']) &
            (galah_apogee['snr_c2_iraf'] > 100) &
            (galah_apogee['flag_'+element+'_fe'] == 0) &
            # good apogee selection
            (galah_apogee['ASPCAPFLAG'] == 0) &
            (galah_apogee[apogee_element+'_FE_FLAG'] == 0) &
            (galah_apogee[apogee_element+'_FE'] > -5) &
            # giants
            (
                (galah_apogee['teff'] <= 5500) &
                (galah_apogee['logg'] <= 3.8)
            )
        )
        
        delta_galah_apogee[element+'_fe'] = r'$'+"{:.2f}".format(np.mean(galah_apogee[element+'_fe'][good_selection] - galah_apogee[apogee_element+'_FE'][good_selection]))+' \pm '+"{:.2f}".format(np.std(galah_apogee[element+'_fe'][good_selection] - galah_apogee[apogee_element+'_FE'][good_selection]))+'$'
        delta_galah_apogee['dwarfs_'+element+'_fe'] = r'$'+"{:.2f}".format(np.mean(galah_apogee[element+'_fe'][dwarf_selection] - galah_apogee[apogee_element+'_FE'][dwarf_selection]))+' \pm '+"{:.2f}".format(np.std(galah_apogee[element+'_fe'][dwarf_selection] - galah_apogee[apogee_element+'_FE'][dwarf_selection]))+'$'
        delta_galah_apogee['giants_'+element+'_fe'] = r'$'+"{:.2f}".format(np.mean(galah_apogee[element+'_fe'][giant_selection] - galah_apogee[apogee_element+'_FE'][giant_selection]))+' \pm '+"{:.2f}".format(np.std(galah_apogee[element+'_fe'][giant_selection] - galah_apogee[apogee_element+'_FE'][giant_selection]))+'$'

In [202]:
def add_sp_row(parameter, parameter_label, unit, html_text, latex_table, galah=galah, asplund=asplund ,apogee=apogee):
    
#     html_text.append(['<TR>'])
#     html_text.append(['<TD style="text-align:center">'+parameter+' <TD style="text-align:center"> '+unit])
#     html_text.append(['<TD style="text-align:center">'+galah[parameter]+'&#177;'+galah['e_'+parameter]+'</TD>'])
#     html_text.append(['<TD style="text-align:center">'+asplund[parameter]+'&#177;'+asplund['e_'+parameter]+'</TD>'])
#     html_text.append(['<TD style="text-align:center">'+apogee[parameter]+'&#177;'+apogee['e_'+parameter]+'</TD>'])

    if galah[parameter] == '-':
        galah_text = '-'
    else:
        galah_text = galah[parameter]+'\pm'+galah['e_'+parameter]
        
    if asplund[parameter] == '-':
        asplund_text = '-'
    else:
        asplund_text = asplund[parameter]+'\pm'+asplund['e_'+parameter]

    if apogee[parameter] == '-':
        apogee_text = '-'
    else:
        apogee_text = apogee[parameter]+'\pm'+apogee['e_'+parameter]
    
    latex_table.append([parameter_label+' & '+unit+' & $'+galah_text+'$ & $'+asplund_text+'$ & $'+apogee_text+'$ \\\\'])
    
    return(html_text, latex_table)


In [203]:
def add_ab_row(parameter, wavelength, html_text, latex_table, galah=galah, asplund=asplund ,apogee=apogee):

    if parameter == 'alpha_fe':
        a_galah_text = galah['A_alpha_fe']
    elif wavelength == 'combined':
        a_galah_text = galah['A_'+parameter[:-3]]
    else:
        a_galah_text = galah['A_'+wavelength]
    
    try:
        if galah[parameter] == '-':
            xfe_galah_text = '-'
        else:
            xfe_galah_text = r'$'+galah[parameter]+' \pm '+galah['e_'+parameter]+'$'
    except:
        xfe_galah_text = 'NA'
        
    solar_circle_selection = (
        (galah_dr3['flag_sp'] == 0) &
        (4500 < galah_dr3['teff']) &
        (galah_dr3['teff'] < 6500) &
        (-0.1 < galah_dr3['fe_h']) &
        (galah_dr3['fe_h'] < 0.1) &
        (galah_dr3['r_est'] < 500) &
        (galah_dr3['snr_c2_iraf'] > 40) &
        (galah_dr3['flag_'+parameter[:-3]+'_fe'] == 0)
    )
    
    dwarfs_solar_circle_selection = (
        (
            (galah_dr3['teff'] > 5500) |
            (galah_dr3['logg'] > 3.8) 
        ) &
        (galah_dr3['flag_sp'] == 0) &
        (4500 < galah_dr3['teff']) &
        (galah_dr3['teff'] < 6500) &
        (-0.1 < galah_dr3['fe_h']) &
        (galah_dr3['fe_h'] < 0.1) &
        (galah_dr3['r_est'] < 500) &
        (galah_dr3['snr_c2_iraf'] > 40) &
        (galah_dr3['flag_'+parameter[:-3]+'_fe'] == 0)
    )
    
    giants_solar_circle_selection = (
        (
            (galah_dr3['teff'] <= 5500) &
            (galah_dr3['logg'] <= 3.8) 
        ) &
        (galah_dr3['flag_sp'] == 0) &
        (4500 < galah_dr3['teff']) &
        (galah_dr3['teff'] < 6500) &
        (-0.1 < galah_dr3['fe_h']) &
        (galah_dr3['fe_h'] < 0.1) &
        (galah_dr3['r_est'] < 500) &
        (galah_dr3['snr_c2_iraf'] > 40) &
        (galah_dr3['flag_'+parameter[:-3]+'_fe'] == 0)
    )
    
    solar_circle = '-'
    if len(galah_dr3[parameter[:-3]+'_fe'][solar_circle_selection])>0:
        print(len(galah_dr3[parameter[:-3]+'_fe'][solar_circle_selection]))
        solar_circle = r'$'+"{:.2f}".format(np.mean(galah_dr3[parameter[:-3]+'_fe'][solar_circle_selection]))+'\pm'+"{:.2f}".format(np.std(galah_dr3[parameter[:-3]+'_fe'][solar_circle_selection]))+'$'

    dwarfs_solar_circle = '-'
    if len(galah_dr3[parameter[:-3]+'_fe'][dwarfs_solar_circle_selection])>0:
        print(len(galah_dr3[parameter[:-3]+'_fe'][dwarfs_solar_circle_selection]))
        dwarfs_solar_circle = r'$'+"{:.2f}".format(np.mean(galah_dr3[parameter[:-3]+'_fe'][dwarfs_solar_circle_selection]))+'\pm'+"{:.2f}".format(np.std(galah_dr3[parameter[:-3]+'_fe'][dwarfs_solar_circle_selection]))+'$'

    giants_solar_circle = '-'
    if len(galah_dr3[parameter[:-3]+'_fe'][giants_solar_circle_selection])>0:
        print(len(galah_dr3[parameter[:-3]+'_fe'][giants_solar_circle_selection]))
        giants_solar_circle = r'$'+"{:.2f}".format(np.mean(galah_dr3[parameter[:-3]+'_fe'][giants_solar_circle_selection]))+'\pm'+"{:.2f}".format(np.std(galah_dr3[parameter[:-3]+'_fe'][giants_solar_circle_selection]))+'$'
        
    # combined runs not in Asplund
    if parameter in ['alpha_fe']:
        asplund_text = '-'
    else:
        asplund_text = r'$'+asplund['A_'+parameter[:-3]]+' \pm '+asplund['e_A_'+parameter[:-3]]+'$'

    try:
        apogee_text = '-'
        if apogee[parameter] != '-':
            if wavelength in ['combined','Li6708','C6588','Mg5711','K7699','Rb7800','Sr6550','Ce4774','Eu6645']:
                apogee_text = r'$'+apogee[parameter]+' \pm '+apogee['e_'+parameter]+'$'    
    except:
        apogee_text = '-'
        
    diff_apogee = '-'
    if wavelength in ['combined','Li6708','C6588','Mg5711','K7699','Rb7800','Sr6550','Ce4774','Eu6645']:
        diff_apogee = delta_galah_apogee[parameter]

    dwarf_diff_apogee = '-'
    if wavelength in ['combined','Li6708','C6588','Mg5711','K7699','Rb7800','Sr6550','Ce4774','Eu6645']:
        dwarf_diff_apogee = delta_galah_apogee['dwarfs_'+parameter]
    giant_diff_apogee = '-'
    if wavelength in ['combined','Li6708','C6588','Mg5711','K7699','Rb7800','Sr6550','Ce4774','Eu6645']:
        giant_diff_apogee = delta_galah_apogee['giants_'+parameter]

    if wavelength != 'combined':
        wavelength = wavelength[-4:]
        
    print(parameter[:-3]+' & '+wavelength+' & '+a_galah_text+' & '+asplund_text+' & '+xfe_galah_text+' & '+solar_circle+' & '+apogee_text+' & '+diff_apogee+' \\\\')
    
    html_text.append(['<TR>'])
    html_text.append(['<TD style="text-align:center"> '+parameter[:-3]]) 
    html_text.append(['<TD style="text-align:center"> '+wavelength]) 
    html_text.append(['<TD style="text-align:center"> '+a_galah_text]) 
    html_text.append(['<TD style="text-align:center"> '+asplund_text]) 
    html_text.append(['<TD style="text-align:center"> '+xfe_galah_text])
    html_text.append(['<TD style="text-align:center"> '+solar_circle])
    html_text.append(['<TD style="text-align:center"> '+dwarfs_solar_circle])
    html_text.append(['<TD style="text-align:center"> '+giants_solar_circle])
    html_text.append(['<TD style="text-align:center"> '+apogee_text])
    html_text.append(['<TD style="text-align:center"> '+diff_apogee])
    html_text.append(['<TD style="text-align:center"> '+dwarf_diff_apogee])
    html_text.append(['<TD style="text-align:center"> '+giant_diff_apogee])

    latex_table.append([
        parameter[:-3]+' & '+wavelength+' & '+a_galah_text+' & '+asplund_text+' & '+xfe_galah_text+' & '+solar_circle+' & '+apogee_text+' & '+diff_apogee+' \\\\'
    ])
    
    return(html_text, latex_table)


## Solar Parameters

In [204]:
html_text = [['<html><body><table border=2>']]
html_text.append(['<TR>'])
html_text.append(['<TD style="text-align:center"> Parameter <TD style="text-align:center"> Unit <TD style="text-align:center"> GALAH DR3 <TD style="text-align:center"> Ramirez+11 <TD style="text-align:center"> APOGEE DR16'])

latex_table = [
    ['\\begin{table*}'],
    ['\\caption{Reference values for Sun from GALAH DR3 (this work), literature, and APOGEE DR16 VESTA \\citep{SDSSDR16}. The literature is a combination of IAU Solar values \citep{Prsa2016}, ages from \citet{Bonanno2002}, velocity estimates ($v_\text{mic}$ and $v_\text{broad}$) from \citet{Jofre2018a}, and abundances from \citet{Asplund2009}. [M/H] is the pseudo-iron abundance sme.feh for GALAH DR3 and \\textsc{m\_h} from SDSS DR16. For APOGEE DR16 we use the a quadratic sum of $v_\\text{macro}$ and $v \sin i$ as $v_\\text{broad}$ value. We use values from the \hyperref{https://data.sdss.org/sas/dr16/apogee/spectro/aspcap/r12/l33/allStar-r12-l33.html}{SDSS website}, computed via [X/Fe] = [X/M] - [Fe/M] for the Vesta abundances of O, Na, V, and Ce.}\label{tab:solar_reference_values1}'],
    ['\\centering'],
    ['\\begin{tabular}{cc|ccc}'],
    ['\\hline'],
    ['Parameter & Unit & GALAH DR3 & Literature & APOGEE DR16 \\\\'],
    ['\\hline'],
]

(html_text, latex_table) = add_sp_row(parameter='teff', parameter_label=r'$T_\text{eff}$', unit='[K]', html_text=html_text, latex_table=latex_table);
(html_text, latex_table) = add_sp_row(parameter='logg', parameter_label=r'$\log g$', unit='[dex]', html_text=html_text, latex_table=latex_table);
(html_text, latex_table) = add_sp_row(parameter='m_h', parameter_label=r'$\mathrm{[M/H]}$', unit='[dex]', html_text=html_text, latex_table=latex_table);
(html_text, latex_table) = add_sp_row(parameter='fe_h', parameter_label=r'$\mathrm{[Fe/H]}$', unit='[dex]', html_text=html_text, latex_table=latex_table);
(html_text, latex_table) = add_sp_row(parameter='mass', parameter_label=r'Mass', unit='[$\mathrm{M_\odot}$]', html_text=html_text, latex_table=latex_table);
(html_text, latex_table) = add_sp_row(parameter='age', parameter_label=r'Age', unit='[Gyr]', html_text=html_text, latex_table=latex_table);
(html_text, latex_table) = add_sp_row(parameter='lbol', parameter_label=r'$L_\text{bol}$', unit=r'$[\mathrm{L_{bol,\odot}}]$', html_text=html_text, latex_table=latex_table);
(html_text, latex_table) = add_sp_row(parameter='vmic', parameter_label=r'$v_\text{mic}$', unit='[km/s]', html_text=html_text, latex_table=latex_table);
(html_text, latex_table) = add_sp_row(parameter='vbroad', parameter_label=r'$v_\text{broad}$', unit='[km/s]', html_text=html_text, latex_table=latex_table);

html_text.append(['</table>'])
html_text.append(['vbroad: (degenerate) combination of vsini and vmacro. For GALAH, it is estimated by SME by setting vmacro=0 and fitting vsini. Ramirez+11 did not give a value. For APOGEE DR16, only VMACRO is reported.'])
html_text.append(['</body></html>'])
latex_table.append(['\\hline'])
latex_table.append(['\\end{tabular}'])
latex_table.append(['\\end{table*}'])

np.savetxt('solar_references1.html',np.array(html_text),fmt='%s')
np.savetxt('solar_references1.tex',np.array(latex_table),fmt='%s')
np.savetxt('../../dr3_release_paper/tables/solar_references1.tex',np.array(latex_table),fmt='%s')


## Solar abundances

In [205]:
html_text = ([['<html><body>']])
             
html_text.append(['<script type="text/x-mathjax-config">MathJax.Hub.Config({tex2jax: {inlineMath: [["$","$"], ["\\(","\\)"]]}});'])
html_text.append(['</script>'])
html_text.append(['<script type="text/javascript" src="http://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.1/MathJax.js?config=TeX-AMS-MML_HTMLorMML">'])
html_text.append(['</script>'])             
html_text.append(['<table border=2>'])
html_text.append(['<TR>'])
html_text.append(['<TD style="text-align:center"> Element'])
html_text.append(['<TD style="text-align:center"> lambda'])
html_text.append(['<TD style="text-align:center"> Solar A_X'])
html_text.append(['<TD style="text-align:center"> A_X'])
html_text.append(['<TD style="text-align:center"> [X/Fe]'])
html_text.append(['<TD style="text-align:center"> [X/Fe]'])
html_text.append(['<TD style="text-align:center"> [X/Fe]'])
html_text.append(['<TD style="text-align:center"> [X/Fe]'])
html_text.append(['<TD style="text-align:center"> [X/Fe]'])
html_text.append(['<TD style="text-align:center"> [X/Fe]'])
html_text.append(['<TD style="text-align:center"> [X/Fe]'])
html_text.append(['<TD style="text-align:center"> [X/Fe]'])

html_text.append(['<TR>'])
html_text.append(['<TD style="text-align:center"> '])
html_text.append(['<TD style="text-align:center"> [AA]'])
html_text.append(['<TD style="text-align:center"> GALAH DR3'])
html_text.append(['<TD style="text-align:center"> Asplund2009'])
html_text.append(['<TD style="text-align:center"> GALAH DR3'])
html_text.append(['<TD style="text-align:center"> GALAH DR3'])
html_text.append(['<TD style="text-align:center"> GALAH DR3'])
html_text.append(['<TD style="text-align:center"> GALAH DR3'])
html_text.append(['<TD style="text-align:center"> APOGEE DR16'])
html_text.append(['<TD style="text-align:center"> APOGEE DR16'])
html_text.append(['<TD style="text-align:center"> APOGEE DR16'])
html_text.append(['<TD style="text-align:center"> APOGEE DR16'])

html_text.append(['<TR>'])
html_text.append(['<TD style="text-align:center"> '])
html_text.append(['<TD style="text-align:center"> '])
html_text.append(['<TD style="text-align:center"> Zero point'])
html_text.append(['<TD style="text-align:center"> '])
html_text.append(['<TD style="text-align:center"> Skyflat'])
html_text.append(['<TD style="text-align:center"> Solar Circle All'])
html_text.append(['<TD style="text-align:center"> Solar Circle Dwarfs'])
html_text.append(['<TD style="text-align:center"> Solar Circle Giants'])
html_text.append(['<TD style="text-align:center"> VESTA'])
html_text.append(['<TD style="text-align:center"> Overlap All'])
html_text.append(['<TD style="text-align:center"> Overlap Dwarfs'])
html_text.append(['<TD style="text-align:center"> Overlap Giants'])

latex_table = [
    ['\\begin{table*}'],
    ['\\caption{Reference values for Sun from GALAH DR3 (this work), \citet{Asplund2009}, and APOGEE DR16 VESTA \\citep{SDSSDR16}. [M/H] is the pseudo-iron abundance sme.feh for GALAH DR3 and \\textsc{m\_h} from SDSS DR16. For APOGEE DR16 we use the a quadratic sum of $v_\\text{macro}$ and $v \sin i$ as $v_\\text{broad}$ value. We use values from the \hyperref{https://data.sdss.org/sas/dr16/apogee/spectro/aspcap/r12/l33/allStar-r12-l33.html}{SDSS website}, computed via [X/Fe] = [X/M] - [Fe/M] for the Vesta abundances of O, Na, V, and Ce.}\label{tab:solar_reference_values2}'],
    ['\\centering'],
    ['\\begin{tabular}{cc|cc|cccc}'],
    ['\\hline'],
    ['Element &  $\lambda$ & $\mathrm{A(X_\odot)}$ & $\mathrm{A(X_\odot)}$ & [X/Fe] & [X/Fe] & [X/Fe] & [X/Fe] \\\\'],
    ['        &   & GALAH DR3 & \citet{Asplund2009} & GALAH DR3 & GALAH DR3 & APOGEE DR16  & APOGEE DR16 \\\\'],
    ['        & [\AA] & Zero point & Photosphere & Skyflat & Solar Circle & VESTA  & Overlap \\\\'],
    ['\\hline'],
]

(html_text, latex_table) = add_ab_row(parameter='alpha_fe', wavelength='combined', html_text=html_text, latex_table=latex_table);
(html_text, latex_table) = add_ab_row(parameter='Li_fe', wavelength='Li6708', html_text=html_text, latex_table=latex_table);
(html_text, latex_table) = add_ab_row(parameter='C_fe', wavelength='C6588', html_text=html_text, latex_table=latex_table);
(html_text, latex_table) = add_ab_row(parameter='O_fe', wavelength='combined', html_text=html_text, latex_table=latex_table);
(html_text, latex_table) = add_ab_row(parameter='Na_fe', wavelength='combined', html_text=html_text, latex_table=latex_table);
(html_text, latex_table) = add_ab_row(parameter='Mg_fe', wavelength='Mg5711', html_text=html_text, latex_table=latex_table);
(html_text, latex_table) = add_ab_row(parameter='Si_fe', wavelength='combined', html_text=html_text, latex_table=latex_table);
(html_text, latex_table) = add_ab_row(parameter='Al_fe', wavelength='combined', html_text=html_text, latex_table=latex_table);
(html_text, latex_table) = add_ab_row(parameter='K_fe', wavelength='K7699', html_text=html_text, latex_table=latex_table);
(html_text, latex_table) = add_ab_row(parameter='Ca_fe', wavelength='combined', html_text=html_text, latex_table=latex_table);
(html_text, latex_table) = add_ab_row(parameter='Sc_fe', wavelength='combined', html_text=html_text, latex_table=latex_table);
(html_text, latex_table) = add_ab_row(parameter='Ti_fe', wavelength='combined', html_text=html_text, latex_table=latex_table);
(html_text, latex_table) = add_ab_row(parameter='Ti_fe', wavelength='Ti4758', html_text=html_text, latex_table=latex_table);
(html_text, latex_table) = add_ab_row(parameter='Ti_fe', wavelength='Ti4759', html_text=html_text, latex_table=latex_table);
(html_text, latex_table) = add_ab_row(parameter='Ti_fe', wavelength='Ti4782', html_text=html_text, latex_table=latex_table);
(html_text, latex_table) = add_ab_row(parameter='Ti_fe', wavelength='Ti4802', html_text=html_text, latex_table=latex_table);
(html_text, latex_table) = add_ab_row(parameter='Ti_fe', wavelength='Ti4820', html_text=html_text, latex_table=latex_table);
(html_text, latex_table) = add_ab_row(parameter='Ti_fe', wavelength='Ti5739', html_text=html_text, latex_table=latex_table);
(html_text, latex_table) = add_ab_row(parameter='Ti2_fe', wavelength='combined', html_text=html_text, latex_table=latex_table);
(html_text, latex_table) = add_ab_row(parameter='Ti2_fe', wavelength='Ti4720', html_text=html_text, latex_table=latex_table);
(html_text, latex_table) = add_ab_row(parameter='Ti2_fe', wavelength='Ti4765', html_text=html_text, latex_table=latex_table);
(html_text, latex_table) = add_ab_row(parameter='Ti2_fe', wavelength='Ti4799', html_text=html_text, latex_table=latex_table);
(html_text, latex_table) = add_ab_row(parameter='Ti2_fe', wavelength='Ti4866', html_text=html_text, latex_table=latex_table);
(html_text, latex_table) = add_ab_row(parameter='V_fe', wavelength='combined', html_text=html_text, latex_table=latex_table);
(html_text, latex_table) = add_ab_row(parameter='V_fe', wavelength='V4797', html_text=html_text, latex_table=latex_table);
(html_text, latex_table) = add_ab_row(parameter='V_fe', wavelength='V4832', html_text=html_text, latex_table=latex_table);
(html_text, latex_table) = add_ab_row(parameter='Cr_fe', wavelength='combined', html_text=html_text, latex_table=latex_table);
(html_text, latex_table) = add_ab_row(parameter='Mn_fe', wavelength='combined', html_text=html_text, latex_table=latex_table);
(html_text, latex_table) = add_ab_row(parameter='Co_fe', wavelength='combined', html_text=html_text, latex_table=latex_table);
(html_text, latex_table) = add_ab_row(parameter='Co_fe', wavelength='Co5647', html_text=html_text, latex_table=latex_table);
(html_text, latex_table) = add_ab_row(parameter='Co_fe', wavelength='Co6490', html_text=html_text, latex_table=latex_table);
(html_text, latex_table) = add_ab_row(parameter='Co_fe', wavelength='Co6632', html_text=html_text, latex_table=latex_table);
(html_text, latex_table) = add_ab_row(parameter='Co_fe', wavelength='Co7713', html_text=html_text, latex_table=latex_table);
(html_text, latex_table) = add_ab_row(parameter='Ni_fe', wavelength='combined', html_text=html_text, latex_table=latex_table);
(html_text, latex_table) = add_ab_row(parameter='Ni_fe', wavelength='Ni5847', html_text=html_text, latex_table=latex_table);
(html_text, latex_table) = add_ab_row(parameter='Ni_fe', wavelength='Ni6586', html_text=html_text, latex_table=latex_table);
(html_text, latex_table) = add_ab_row(parameter='Cu_fe', wavelength='combined', html_text=html_text, latex_table=latex_table);
(html_text, latex_table) = add_ab_row(parameter='Cu_fe', wavelength='Cu5700', html_text=html_text, latex_table=latex_table);
(html_text, latex_table) = add_ab_row(parameter='Cu_fe', wavelength='Cu5782', html_text=html_text, latex_table=latex_table);
(html_text, latex_table) = add_ab_row(parameter='Zn_fe', wavelength='combined', html_text=html_text, latex_table=latex_table);
(html_text, latex_table) = add_ab_row(parameter='Zn_fe', wavelength='Zn4722', html_text=html_text, latex_table=latex_table);
(html_text, latex_table) = add_ab_row(parameter='Zn_fe', wavelength='Zn4811', html_text=html_text, latex_table=latex_table);
(html_text, latex_table) = add_ab_row(parameter='Rb_fe', wavelength='Rb7800', html_text=html_text, latex_table=latex_table);
(html_text, latex_table) = add_ab_row(parameter='Sr_fe', wavelength='Sr6550', html_text=html_text, latex_table=latex_table);
(html_text, latex_table) = add_ab_row(parameter='Y_fe', wavelength='combined', html_text=html_text, latex_table=latex_table);
(html_text, latex_table) = add_ab_row(parameter='Y_fe', wavelength='Y4855', html_text=html_text, latex_table=latex_table);
(html_text, latex_table) = add_ab_row(parameter='Y_fe', wavelength='Y4884', html_text=html_text, latex_table=latex_table);
(html_text, latex_table) = add_ab_row(parameter='Zr_fe', wavelength='combined', html_text=html_text, latex_table=latex_table);
(html_text, latex_table) = add_ab_row(parameter='Zr_fe', wavelength='Zr4739', html_text=html_text, latex_table=latex_table);
(html_text, latex_table) = add_ab_row(parameter='Zr_fe', wavelength='Zr4772', html_text=html_text, latex_table=latex_table);
(html_text, latex_table) = add_ab_row(parameter='Zr_fe', wavelength='Zr4806', html_text=html_text, latex_table=latex_table);
(html_text, latex_table) = add_ab_row(parameter='Zr_fe', wavelength='Zr4828', html_text=html_text, latex_table=latex_table);
(html_text, latex_table) = add_ab_row(parameter='Zr_fe', wavelength='Zr5681', html_text=html_text, latex_table=latex_table);
(html_text, latex_table) = add_ab_row(parameter='Mo_fe', wavelength='combined', html_text=html_text, latex_table=latex_table);
(html_text, latex_table) = add_ab_row(parameter='Mo_fe', wavelength='Mo5858', html_text=html_text, latex_table=latex_table);
(html_text, latex_table) = add_ab_row(parameter='Mo_fe', wavelength='Mo6619', html_text=html_text, latex_table=latex_table);

latex_table.append(['\\hline'])
latex_table.append(['\\end{tabular}'])
latex_table.append(['\\end{table*}'])
latex_table.append(['\\begin{table*}'])
latex_table.append(['\\caption{Continuation of Table~\\ref{tab:solar_reference_values2}}'])
latex_table.append(['\\centering'])
latex_table.append(['\\begin{tabular}{cc|cc|cccc}'])
latex_table.append(['\\hline'])
latex_table.append(['Element &  $\lambda$ & $\mathrm{A(X_\odot)}$ & $\mathrm{A(X_\odot)}$ & [X/Fe] & [X/Fe] & [X/Fe] & [X/Fe] \\\\'])
latex_table.append(['        &   & GALAH DR3 & \citet{Asplund2009} & GALAH DR3 & GALAH DR3 & APOGEE DR16  & APOGEE DR16 \\\\'])
latex_table.append(['        & [\AA] & Zero point & \citet{Asplund2009} & Skyflat & Solar Circle & VESTA  & Overlap \\\\'])
latex_table.append(['\\hline'])

(html_text, latex_table) = add_ab_row(parameter='Ru_fe', wavelength='combined', html_text=html_text, latex_table=latex_table);
(html_text, latex_table) = add_ab_row(parameter='Ru_fe', wavelength='Zr4739', html_text=html_text, latex_table=latex_table);
(html_text, latex_table) = add_ab_row(parameter='Ru_fe', wavelength='Zr4739', html_text=html_text, latex_table=latex_table);
(html_text, latex_table) = add_ab_row(parameter='Ba_fe', wavelength='combined', html_text=html_text, latex_table=latex_table);
(html_text, latex_table) = add_ab_row(parameter='La_fe', wavelength='combined', html_text=html_text, latex_table=latex_table);
(html_text, latex_table) = add_ab_row(parameter='La_fe', wavelength='La4749', html_text=html_text, latex_table=latex_table);
(html_text, latex_table) = add_ab_row(parameter='La_fe', wavelength='La4804', html_text=html_text, latex_table=latex_table);
(html_text, latex_table) = add_ab_row(parameter='La_fe', wavelength='La5806', html_text=html_text, latex_table=latex_table);
(html_text, latex_table) = add_ab_row(parameter='Ce_fe', wavelength='Ce4774', html_text=html_text, latex_table=latex_table);
(html_text, latex_table) = add_ab_row(parameter='Nd_fe', wavelength='combined', html_text=html_text, latex_table=latex_table);
(html_text, latex_table) = add_ab_row(parameter='Nd_fe', wavelength='Nd4811', html_text=html_text, latex_table=latex_table);
(html_text, latex_table) = add_ab_row(parameter='Nd_fe', wavelength='Nd5812', html_text=html_text, latex_table=latex_table);
(html_text, latex_table) = add_ab_row(parameter='Sm_fe', wavelength='combined', html_text=html_text, latex_table=latex_table);
(html_text, latex_table) = add_ab_row(parameter='Sm_fe', wavelength='Sm4720', html_text=html_text, latex_table=latex_table);
(html_text, latex_table) = add_ab_row(parameter='Sm_fe', wavelength='Sm4848', html_text=html_text, latex_table=latex_table);
(html_text, latex_table) = add_ab_row(parameter='Eu_fe', wavelength='Eu6645', html_text=html_text, latex_table=latex_table);

html_text.append(['</table>'])
html_text.append(['Giants: Teff < 5500 K | logg < 3.8, Dwarfs: ~Giants. <br>'])
html_text.append(['vbroad: (degenerate) combination of vsini and vmacro. <br>'])
html_text.append(['For GALAH, it is estimated by SME by setting vmacro=0 and fitting vsini. <br>'])
html_text.append(['Ramirez+11 did not give a value. <br>'])
html_text.append(['For APOGEE DR16, only VMACRO is reported. <br>'])
html_text.append(['</body></html>'])

latex_table.append(['\\hline'])
latex_table.append(['\\end{tabular}'])
latex_table.append(['\\end{table*}'])

np.savetxt('solar_references2.html',np.array(html_text),fmt='%s')
np.savetxt('solar_references2.tex',np.array(latex_table),fmt='%s')
np.savetxt('../../dr3_release_paper/tables/solar_references2.tex',np.array(latex_table),fmt='%s')

17153
16303
850
alpha & combined & - & - & $-0.00 \pm 0.02$ & $0.01\pm0.05$ & $-0.01 \pm 0.01$ & $0.01 \pm 0.07$ \\
6566
6436
130
Li & 6708 & 1.05 & $1.05 \pm 0.10$ & - & $1.27\pm0.39$ & - & - \\
3117
3115
2
C & 6588 & 8.45 & $8.43 \pm 0.05$ & - & $0.04\pm0.09$ & $0.02 \pm 0.02$ & $-0.02 \pm 0.11$ \\
16074
15240
834
O & combined & 8.82 & $8.69 \pm 0.05$ & $-0.17 \pm 0.06$ & $-0.00\pm0.14$ & $0.05 \pm -$ & $0.09 \pm 0.22$ \\
17186
16336
850
Na & combined & 6.17 & $6.24 \pm 0.04$ & $-0.10 \pm 0.02$ & $-0.11\pm0.10$ & $-0.01 \pm -$ & $-0.03 \pm 0.19$ \\
16674
15827
847
Mg & 5711 & 7.60 & $7.60 \pm 0.04$ & $0.00 \pm 0.03$ & $0.01\pm0.08$ & $-0.00 \pm 0.01$ & $-0.00 \pm 0.10$ \\
17152
16302
850
Si & combined & 7.47 & $7.51 \pm 0.03$ & $-0.02 \pm 0.03$ & $0.00\pm0.06$ & $-0.00 \pm 0.01$ & $0.04 \pm 0.13$ \\
17117
16266
851
Al & combined & 6.43 & $6.45 \pm 0.03$ & $-0.02 \pm 0.03$ & $0.02\pm0.09$ & $0.01 \pm 0.02$ & $0.08 \pm 0.16$ \\
16786
15954
832
K & 7699 & 5.07 & $5.03 \pm 0.09$ & $-0.09

# Stellar parameters Synthesis

In [ ]:
def plot_sp_segments(html_text, create_new=False):
    
    single_sme = get_mode_sme(single_spectrum, mode='Sp')
    combined_sme = get_mode_sme(combined_spectrum, mode='Sp')

    html_text.append(['<TR><TD>Parameter<TD>Single<TD>Combined'])
    html_text.append(['<TR><TD>TEFF<TD>'+str(single_sme.TEFF)+'<TD>'+str(combined_sme.TEFF)])
    html_text.append(['<TR><TD>LOGG<TD>'+str(single_sme.GRAV)+'<TD>'+str(combined_sme.GRAV)])
    html_text.append(['<TR><TD>FEH<TD>'+str(single_sme.FEH)+'<TD>'+str(combined_sme.FEH)])
    html_text.append(['<TR><TD>VMIC<TD>'+str(single_sme.VMIC)+'<TD>'+str(combined_sme.VMIC)])
    html_text.append(['<TR><TD>VSINI<TD>'+str(single_sme.VSINI)+'<TD>'+str(combined_sme.VSINI)])
    html_text.append(['</table>'])
    html_text.append(['<table>'])
    
    for each_segment_id in range(single_sme.NSEG):
        
        if create_new:
            if each_segment_id == 0:
                pixels_in_segment_single = np.arange(0,single_sme.WIND[0]+1)
                pixels_in_segment_combined = np.arange(0,combined_sme.WIND[0]+1)
            else:
                pixels_in_segment_single = np.arange(single_sme.WIND[each_segment_id-1]+1,single_sme.WIND[each_segment_id]+1)
                pixels_in_segment_combined = np.arange(combined_sme.WIND[each_segment_id-1]+1,combined_sme.WIND[each_segment_id]+1)

            f, ax = plt.subplots(figsize=(15,5))

            ax.errorbar(
                single_sme.WAVE[pixels_in_segment_single],
                single_sme.SOB[pixels_in_segment_single],
                yerr=single_sme.UOB[pixels_in_segment_single],
                label='Single SOB',
                ms=3,c='k'#,fmt='o'
            )
            ax.errorbar(
                combined_sme.WAVE[pixels_in_segment_combined],
                combined_sme.SOB[pixels_in_segment_combined],
                yerr=combined_sme.UOB[pixels_in_segment_combined],
                label='Combined SOB',
                ms=3,c='b'#,fmt='o'
            )
            ax.plot(
                single_sme.WAVE[pixels_in_segment_single],
                single_sme.SMOD[pixels_in_segment_single],
                label='Single SMOD',c='C1'
            )
            ax.plot(
                combined_sme.WAVE[pixels_in_segment_combined],
                combined_sme.SMOD[pixels_in_segment_combined],
                label='Combined SMOD',c='r'
            )

            for mask_line in np.where(single_sme.MOB[pixels_in_segment_single]==1)[0]:
                mask_point = pixels_in_segment_single[mask_line]
                dwave1 = single_sme.WAVE[mask_point] - 0.5*(single_sme.WAVE[mask_point]-single_sme.WAVE[mask_point-1])
                dwave2 = single_sme.WAVE[mask_point] + 0.5*(single_sme.WAVE[mask_point]-single_sme.WAVE[mask_point-1])

                ax.axvspan(dwave1,dwave2,facecolor='yellow',zorder=1)

            ax.legend()

            ax.set_ylim(-0.1,1.1)

            ax.set_xlabel(r'Wavelength [$\AA$]')
            ax.set_ylabel('Flux [norm]')
            ax.set_title('SP Estimation: Single/Combined Spectra for Segment '+str(each_segment_id))

            plt.tight_layout()

            plt.savefig('solar_abundances/sp_spectra/solar_sp_spectra_segm_'+str(each_segment_id)+'.png',dpi=200,bbox_inches='tight')

            plt.close()
        
        html_text.append(['<TR><TD>Segment '+str(each_segment_id)+'<br>'])
        html_text.append(['<TD><a href=solar_abundances/sp_spectra/solar_sp_spectra_segm_'+str(each_segment_id)+'.png><IMG src=solar_abundances/sp_spectra/solar_sp_spectra_segm_'+str(each_segment_id)+'.png width=1024></a>'])

    return(html_text)

In [218]:
html_text = [
    ['<html><body>'],
    ['<table>']
]
html_text = plot_sp_segments(html_text)
html_text.append(['</table>'])
html_text.append(['</body></html>'])
np.savetxt('solar_abundance_synthesis_sp.html',np.array(html_text),fmt='%s')

# Element Abundances

In [255]:
mode = np.genfromtxt('../../analysis/abundances/mode_DR3',usecols=(0,),dtype=str,comments=';')
indexes = np.unique(mode, return_index=True)[1]
mode = [mode[index] for index in sorted(indexes)][2:]

In [282]:
def plot_ab_segments(html_text, mode, create_new=False):
    
    for each_element in mode:
        
        try:
            has_single_mode = True
            if each_element != 'Li':
                single_sme = get_mode_sme(single_spectrum, mode=each_element)
            else:
                single_sme = get_mode_sme(single_spectrum, mode='Li')
        except:
            has_single_mode = False
        try:
            has_combined_mode = True
            combined_sme = get_mode_sme(combined_spectrum, mode=each_element)
        except:
            has_combined_mode = False

        for each_segment_id in range(single_sme.NSEG):
            if create_new:
                if each_segment_id == 0:
                    if has_single_mode: pixels_in_segment_single = np.arange(0,single_sme.WIND[0]+1)
                    if has_combined_mode: pixels_in_segment_combined = np.arange(0,combined_sme.WIND[0]+1)
                else:
                    if has_single_mode: pixels_in_segment_single = np.arange(single_sme.WIND[each_segment_id-1]+1,single_sme.WIND[each_segment_id]+1)
                    if has_combined_mode: pixels_in_segment_combined = np.arange(combined_sme.WIND[each_segment_id-1]+1,combined_sme.WIND[each_segment_id]+1)

                f, ax = plt.subplots(figsize=(15,5))

                if has_single_mode: 
                    ax.errorbar(
                        single_sme.WAVE[pixels_in_segment_single],
                        single_sme.SOB[pixels_in_segment_single],
                        yerr=single_sme.UOB[pixels_in_segment_single],
                        label='Single SOB',
                        fmt='o',ms=3,c='k'
                    )
                if has_combined_mode: 
                    ax.errorbar(
                        combined_sme.WAVE[pixels_in_segment_combined],
                        combined_sme.SOB[pixels_in_segment_combined],
                        yerr=combined_sme.UOB[pixels_in_segment_combined],
                        label='Combined SOB',
                        fmt='o',ms=3,c='b'
                    )
                if has_single_mode: 
                    ax.plot(
                        single_sme.WAVE[pixels_in_segment_single],
                        single_sme.SMOD[pixels_in_segment_single],
                        label='Single SMOD',c='C1'
                    )
                if has_combined_mode: 
                    ax.plot(
                        combined_sme.WAVE[pixels_in_segment_combined],
                        combined_sme.SMOD[pixels_in_segment_combined],
                        label='Combined SMOD',c='r'
                    )

                if has_single_mode: 
                    for mask_line in np.where(single_sme.MOB[pixels_in_segment_single]==1)[0]:
                        mask_point = pixels_in_segment_single[mask_line]
                        dwave1 = single_sme.WAVE[mask_point] - 0.5*(single_sme.WAVE[mask_point]-single_sme.WAVE[mask_point-1])
                        dwave2 = single_sme.WAVE[mask_point] + 0.5*(single_sme.WAVE[mask_point]-single_sme.WAVE[mask_point-1])

                        ax.axvspan(dwave1,dwave2,facecolor='yellow',zorder=1)

                ax.legend()

                ax.set_ylim(-0.1,1.1)

                ax.set_xlabel(r'Wavelength [$\AA$]')
                ax.set_ylabel('Flux [norm]')
                ax.set_title(each_element+' Estimation: Single/Combined Spectra for Segment '+str(each_segment_id))

                plt.tight_layout()

                plt.savefig('solar_abundances/ab_spectra/solar_ab_spectra_'+each_element+'_segm_'+str(each_segment_id)+'.png',dpi=200,bbox_inches='tight')

                plt.close()

            html_text.append(['<TR><TD>'])
            html_text.append(['Element '+each_element+' Segment: '+str(each_segment_id)+'<br>'])
                  
            if has_single_mode:
                free_ab_single = np.where(single_sme.AB_FREE==1)[0]
                if len(free_ab_single) > 0:
                    abund = np.array(single_sme.ABUND)
                    abund[2:] += single_sme.FEH
                    abund[1:] = 10**(abund[1:])
                    A_ABUND = np.log10(abund / abund[0]) + 12
                    html_text.append(['Single: A(X) = '+str(A_ABUND[free_ab_single[0]])+' <br>'])
                else:
                    html_text.append(['Single: A(X) = NaN <br>'])
            else:
                html_text.append(['Single: A(X) = NaN <br>'])

            if has_combined_mode:
                free_ab_combined = np.where(combined_sme.AB_FREE==1)[0]
                if len(free_ab_combined) > 0:
                    abund = np.array(combined_sme.ABUND)
                    abund[2:] += combined_sme.FEH
                    abund[1:] = 10**(abund[1:])
                    A_ABUND = np.array(np.log10(abund / abund[0]) + 12)
                    html_text.append(['Combined: A(X) = '+str(A_ABUND[free_ab_combined[0]])+' <br>'])
                else:
                    html_text.append(['Combined: A(X) = NaN <br>'])
            else:
                html_text.append(['Combined: A(X) = NaN <br>'])               
                
            html_text.append(['<TD><a href=solar_abundances/ab_spectra/solar_ab_spectra_'+each_element+'_segm_'+str(each_segment_id)+'.png><IMG src=solar_abundances/ab_spectra/solar_ab_spectra_'+each_element+'_segm_'+str(each_segment_id)+'.png width=1024></a>'])

    return(html_text)

In [283]:
html_text = [
    ['<html><body>'],
    ['<table>']
]
html_text = plot_ab_segments(html_text, mode=mode, create_new=True)
html_text.append(['</table>'])
html_text.append(['</body></html>'])
np.savetxt('solar_abundance_synthesis_ab.html',np.array(html_text),fmt='%s')

No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
